<a href="https://colab.research.google.com/github/Wahyu-triu/Mobile-JKN-Sentiment-Analysis/blob/main/Scraping_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Library

In [ ]:
# install google-play-scarper
!pip install google-play-scraper

     |████████████████████████████████| 52 kB 586 kB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-1.0.1-py3-none-any.whl size=24191 sha256=2f211880bb574888776969dcf267d86484e00c461f7c13e75ab227fc0b58df60
  Stored in directory: /root/.cache/pip/wheels/05/8f/af/b45bcc354aac4f16e943658c14517ca439f5b18b766361a260
Successfully built google-play-scraper


Import Packages

In [2]:
# import packages
#from google_play_scraper import app
import pandas as pd
import numpy as np

In [ ]:
# scrap review from indonesia market

from google_play_scraper import Sort, reviews_all

in_reviews = reviews_all(
    'app.bpjs.mobile', 
    sleep_milliseconds=0, # defaults to 0
    lang='in', # defaults to 'en'
    country='in', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

### Put review into pandas DataFrame

In [ ]:
in_reviews

In [ ]:
# made it into 1 column
df = pd.DataFrame(np.array(in_reviews), columns=['review'])

In [ ]:
# split values based on columns name
df = df.join(pd.DataFrame(df.pop('review').tolist()))

In [ ]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOHEIujNLZHf58zzjsy5FcR77AzXT1qJnC6Ex1G...,NONOK NONOK,https://play-lh.googleusercontent.com/a-/AOh14...,sangat bermanfaat,5,0,3.6.1,2021-08-26 02:50:07,None,NaT
1,gp:AOqpTOGWyX88zWtpGqHoVI4_vIkDPzmy_5UuAADR3_q...,Ari Pambudi,https://play-lh.googleusercontent.com/a-/AOh14...,dikit dikit update,3,0,3.5.0,2021-08-26 02:46:35,None,NaT
2,gp:AOqpTOExiYJux9v1bMQNCvpHWhEwuuo7IK0d140HYni...,Musim Rebah,https://play-lh.googleusercontent.com/a-/AOh14...,Saya sudah mendownload apk JKN mobile. Lalu me...,1,0,3.6.1,2021-08-26 02:43:46,None,NaT
3,gp:AOqpTOEuVlyw5wIhLXkxsEO3_BQDqzOggT_cgvdmBXh...,Par Yanti,https://play-lh.googleusercontent.com/a/AATXAJ...,Blm di pake,1,0,3.5.0,2021-08-26 02:38:19,None,NaT
4,gp:AOqpTOF5T6OUaAKmz_Nl1WL6UZNDsqomi45me6vbKxQ...,siti Rani,https://play-lh.googleusercontent.com/a-/AOh14...,Ok,5,0,None,2021-08-26 02:34:11,None,NaT


In [ ]:
df_try = pd.DataFrame(np.array(in_reviews), columns=['review'])
df_try.head()

,review
0,{'reviewId': 'gp:AOqpTOHEIujNLZHf58zzjsy5FcR77...
1,{'reviewId': 'gp:AOqpTOGWyX88zWtpGqHoVI4_vIkDP...
2,{'reviewId': 'gp:AOqpTOExiYJux9v1bMQNCvpHWhEwu...
3,{'reviewId': 'gp:AOqpTOEuVlyw5wIhLXkxsEO3_BQDq...
4,{'reviewId': 'gp:AOqpTOF5T6OUaAKmz_Nl1WL6UZNDs...


In [ ]:
df_try.pop('review').tolist()

## Pre-processing

In [ ]:
# feature selections

df.head(2)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOHEIujNLZHf58zzjsy5FcR77AzXT1qJnC6Ex1G...,NONOK NONOK,https://play-lh.googleusercontent.com/a-/AOh14...,sangat bermanfaat,5,0,3.6.1,2021-08-26 02:50:07,None,NaT
1,gp:AOqpTOGWyX88zWtpGqHoVI4_vIkDPzmy_5UuAADR3_q...,Ari Pambudi,https://play-lh.googleusercontent.com/a-/AOh14...,dikit dikit update,3,0,3.5.0,2021-08-26 02:46:35,None,NaT


Karena tujuannya adalah hendak melukan sentiment analysis tentang review pengguna mobile JKN, maka saya hanya akan memilih beberapa column:
- Content
- Score

In [ ]:
df_sent = df[['content', 'score']]
df_sent = df_sent.rename(columns = {'content': 'review','score':'rating'}).reset_index()
df_sent = df_sent.rename(columns = {'index':'id'})
df_sent.head(3)

,id,review,rating
0,0,sangat bermanfaat,5
1,1,dikit dikit update,3
2,2,Saya sudah mendownload apk JKN mobile. Lalu me...,1


### Save file into csv

In [3]:
df_sent.to_csv('data review Mobile JKN.csv')

NameError: ignored

### Read CSV


In [4]:
df_sent = pd.read_csv('/content/drive/MyDrive/Scraping/hasil/data review Mobile JKN.csv')

### Continue pre-processing

In [5]:
# check df shape
df_sent.shape

(122070, 4)

In [6]:
# check missing value
df_sent.isnull().sum()

Unnamed: 0    0
id            0
review        1
rating        0
dtype: int64

#### Handling Missing value

In [7]:
# handling missing value
# saya putuskan untuk drop rows karena hanya ada 1 missing value

# drop missing value
df_sent = df_sent.dropna(how='any')

In [8]:
df_sent.isnull().sum()

# sudah tidak ada missing value

Unnamed: 0    0
id            0
review        0
rating        0
dtype: int64

In [9]:
# check comment with rating 3
df_sent.loc[df_sent['rating']==3]

,Unnamed: 0,id,review,rating
1,1,1,dikit dikit update,3
18,18,18,Kok muter2,3
57,57,57,Ini kenapa ga bisa pakai bank BNI ya ? Kan di ...,3
62,62,62,Update terus,3
76,76,76,"update supaya bisa ubah NIK KTP,,soalnya suka ...",3
...,...,...,...,...
122029,122029,122029,Akhirnya bisa jg di update,3
122035,122035,122035,Ada peningkatan 😉,3
122043,122043,122043,Tiga dl ya,3
122046,122046,122046,Tidak bisa registrasi menggunakan apk ini. Sem...,3


#### Change rating to sentiment data

- yang saya maksud data sentiment adalah sentimen + dan negatif, saya tidak memakai netral. 
- sehingga bentuk perubahannya adalah untuk score 4 - 5 (tinggi) akan saya ganti menjadi POSITIF.
- sedangkan untuk score 1 - 2 akan saya ubah menjadi NEGATIF.
- untuk score dengan value 3 akan saya drop rows. karena saya rasa itu nilai tengah, dan ketika saya cek real commentnya ada yang bernada positif ada yang bernada keluhan. sehingga agar tidak membingungkan saya drop rows.

In [10]:
# make a keeprows

keeprows = []
for x, baris in df_sent.iterrows():
  if baris['rating'] != 3:
    keeprows.append(baris)
  else:
    continue
  
df_sent = pd.DataFrame(keeprows)
df_sent.head(3)

,Unnamed: 0,id,review,rating
0,0,0,sangat bermanfaat,5
2,2,2,Saya sudah mendownload apk JKN mobile. Lalu me...,1
3,3,3,Blm di pake,1


In [11]:
df_sent.drop(['Unnamed: 0', 'id'], axis=1, inplace=True)
df_sent = df_sent.reset_index()
df_sent.drop('index', axis=1, inplace=True)
df_sent.head(2)

,review,rating
0,sangat bermanfaat,5
1,Saya sudah mendownload apk JKN mobile. Lalu me...,1


In [12]:
# cek unique value in column rating
df_sent['rating'].value_counts()

5    55596
1    39159
4    10615
2     7834
Name: rating, dtype: int64

#### Transform the numeric value

In [13]:
# transform value into sentiment 

value = []
for x, baris in df_sent.iterrows():
  if baris['rating'] == 1:
    baris['rating'] = 'NEGATIF'
    value.append(baris)
  elif baris['rating'] == 2:
    baris['rating'] = 'NEGATIF'
    value.append(baris)
  elif baris['rating'] == 4:
    baris['rating'] = 'POSITIF'
    value.append(baris)
  elif baris['rating'] == 5:
    baris['rating'] = 'POSITIF'
    value.append(baris)
  else:
    continue

df_sent = pd.DataFrame(value)
df_sent.head(3)

,review,rating
0,sangat bermanfaat,POSITIF
1,Saya sudah mendownload apk JKN mobile. Lalu me...,NEGATIF
2,Blm di pake,NEGATIF


In [14]:
# cek unique value in column rating
df_sent['rating'].value_counts()

POSITIF    66211
NEGATIF    46993
Name: rating, dtype: int64

### Imbalance Data:
- Perbandingan antara rating positif dan negatif sekitar 6:4.
- Dan 6:4 saya rasa sudah balance. karena tidak lebih dari 70:30


In [15]:
df_sent = df_sent.rename(columns={'rating':'sentiment'})
df_sent.head(3)

,review,sentiment
0,sangat bermanfaat,POSITIF
1,Saya sudah mendownload apk JKN mobile. Lalu me...,NEGATIF
2,Blm di pake,NEGATIF


# Text Pre-Processing Phase

#### Import Library

In [21]:
import re
import string

from string import punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('indonesian')

# Library for features extractions and modeling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm, naive_bayes
from sklearn.linear_model import LogisticRegression

# Sastrawi
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Evaluation metrics
from sklearn.metrics import accuracy_score, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Preprocessing Function

def preprocessing(sentence):
    
    #lowecasing
    sentence = sentence.lower()    
    
    #remove white spaces
    sentence = sentence.strip()
    
    #tokenization
    words = sentence.split()
    
    #remove punctuation/ special character
    remove_table = str.maketrans("", "", punctuation)
    words = [x.translate(remove_table) for x in words]
    
    #remove nonalphanumeric <=3 chars
    words = [x for x in words if x.isalnum()]
    
    #remove stopwords
    words = [x for x in words if x not in stop_words]

    #rejoining the words
    sentence = " ".join(words)
    
    return sentence

In [18]:
# apply functions

df_sent['review'] = [preprocessing(abc) for abc in df_sent['review']]

In [19]:
df_sent

,review,sentiment
0,bermanfaat,POSITIF
1,mendownload apk jkn mobile mengikuti tuntunan ...,NEGATIF
2,blm pake,NEGATIF
3,ok,POSITIF
4,masuk gagal daftar mengisi no hp mencoba hubun...,NEGATIF
...,...,...
113199,mudah membantu terima kasih bpjs kesehatan,POSITIF
113200,membantu peserta chek status kepesertaan bpjs ...,POSITIF
113201,membantu,POSITIF
113202,membantu,POSITIF


### Save clean (after text pre-processing) data to csv

In [20]:
df_sent.to_csv('dfresik.csv')